In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import json
import requests
import geopandas as gpd
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression

# 배경

## 인구이동

In [2]:
path_1 ='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/move.xlsx'

df_1 = pd.read_excel(path_1)
df_1

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,시점,행정구역(시군구)별,총전입 (명),총전출 (명),순이동 (명),시도간전입 (명),시도간전출 (명)
0,2014,서울특별시,1573594,1661425,-87831,475571,563402
1,2014,부산광역시,485710,500802,-15092,132428,147520
2,2014,대구광역시,350213,365739,-15526,97571,113097
3,2014,인천광역시,452297,442973,9324,156922,147598
4,2014,광주광역시,243066,246043,-2977,78745,81722
...,...,...,...,...,...,...,...
165,2023,전라북도,188788,193245,-4457,53965,58422
166,2023,전라남도,183557,185357,-1800,71675,73475
167,2023,경상북도,254287,262492,-8205,107068,115273
168,2023,경상남도,300309,316609,-16300,103910,120210


In [3]:
df_1.rename(columns={'행정구역(시군구)별':'도시',
                   '순이동 (명)': '순이동'}, inplace=True)
df_1['지역']='비수도권'
df_1.loc[df_1['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'
df_1=df_1[['시점','지역', '도시','순이동']]
df_1

,시점,지역,도시,순이동
0,2014,수도권,서울특별시,-87831
1,2014,비수도권,부산광역시,-15092
2,2014,비수도권,대구광역시,-15526
3,2014,수도권,인천광역시,9324
4,2014,비수도권,광주광역시,-2977
...,...,...,...,...
165,2023,비수도권,전라북도,-4457
166,2023,비수도권,전라남도,-1800
167,2023,비수도권,경상북도,-8205
168,2023,비수도권,경상남도,-16300


In [4]:
df_1_group = df_1.groupby(['지역','시점'])['순이동'].sum().reset_index()
df_1_group

,지역,시점,순이동
0,비수도권,2014,21111
1,비수도권,2015,32950
2,비수도권,2016,863
3,비수도권,2017,-16006
4,비수도권,2018,-59797
5,비수도권,2019,-82741
6,비수도권,2020,-87775
7,비수도권,2021,-55697
8,비수도권,2022,-36643
9,비수도권,2023,-46869


In [5]:
df_1_pivot = df_1_group.pivot_table(values='순이동', index='시점', columns='지역')
df_1_pivot_sum = df_1_pivot.cumsum()

In [6]:
# 연도별 수도권 인구이동
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_1_group['시점'],
    y=df_1_group[df_1_group['지역']=='수도권']['순이동'],
    name='인구이동',
    marker_color='rgb(150,150,450)',
    text=round(df_1_group[df_1_group['지역']=='수도권']['순이동']/1000, 1)
))
fig.add_trace(go.Scatter(
    x=df_1_group['시점'],
    y=df_1_group[df_1_group['지역']=='수도권']['순이동'],
    mode='lines+markers',
    name='인구이동',
    marker_color='rgb(100,100,300)'
))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)
fig.update_layout(showlegend=True)

fig.show()

In [7]:
# 연도별 수도권 누적 인구이동
fig = go.Figure()

fig.add_trace(go.Bar(
    x=df_1_pivot_sum.index,
    y=df_1_pivot_sum['수도권'],
    name='누적 인구이동',
    marker_color='rgb(150,150,450)',
    text=round(df_1_pivot_sum['수도권']/1000,1)
))

fig.add_trace(go.Scatter(
    x=df_1_pivot_sum.index,
    y=df_1_pivot_sum['수도권'],
    mode='lines+markers',
    name='누적 인구이동',
    marker_color='rgb(100,100,300)'
))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

## 인구분포

In [8]:
path_2='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/people.xlsx'

df_2 = pd.read_excel(path_2)
df_2.head()

,시점,행정구역(시군구)별,성별,계,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
0,2023.0,서울특별시,계,9334933.5,220265.5,308678.0,357808.5,381243.0,564693.5,802843.5,...,588956.0,409781.0,311120.5,372713.5,222881.5,149832.0,105271.0,35320.5,7997.0,1243.5
1,2023.0,부산광역시,계,3291326,78305.5,121997.0,133933.5,130393.0,177426.5,208199.5,...,258134.5,184261.0,129523.0,154773.0,94504.0,60269.0,43269.5,13677.5,2927.5,394.5
2,2023.0,대구광역시,계,2372562,59547.0,94764.0,106649.0,109453.5,137540.5,150989.0,...,157212.0,113543.0,77986.0,106327.5,62908.5,43419.0,31388.5,9946.0,1837.5,247.0
3,2023.0,인천광역시,계,2966979,84134.0,121382.0,137201.0,134775.0,163436.0,205728.5,...,179728.5,110819.0,81399.0,106370.5,60135.5,46235.0,31607.0,11530.5,2665.5,432.0
4,2023.0,광주광역시,계,1420562,40702.5,62653.5,73565.0,75537.0,93358.0,100970.0,...,76434.0,55494.0,41910.0,54074.0,31736.5,22337.5,15645.5,5395.5,1114.0,182.5


In [9]:
df_2=df_2.drop(columns=['성별','시점'])

In [10]:
df_2=df_2.rename(columns={'행정구역(시군구)별':'도시'})

In [11]:
df_2['지역']='비수도권'
df_2.loc[df_2['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'

In [12]:
df_2.insert(1, '지역', df_2.pop('지역'))

In [13]:
url = 'https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/SIDO_MAP_2022.json'
response = requests.get(url)

if response.status_code == 200:
    geo_json = json.loads(response.text)  # 문자열 형태로 변환 후에 파싱합니다.
    # geo_json 이용한 작업 수행
else:
    print("데이터를 가져오는 데 실패했습니다:", response.status_code)


In [14]:
#with open('https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/SIDO_MAP_2022.json') as response:
 #   geo_json = json.load(response)

In [15]:
geo_df = gpd.GeoDataFrame.from_features(geo_json['features'])
geo_df.head()

,geometry,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM
0,"POLYGON ((126.98400 37.63600, 126.94800 37.657...",11,Seoul,서울특별시
1,"POLYGON ((129.28800 35.32100, 129.26300 35.386...",26,Busan,부산광역시
2,"POLYGON ((128.47300 35.83300, 128.47000 35.806...",27,Daegu,대구광역시
3,"MULTIPOLYGON (((126.34300 37.64400, 126.37500 ...",28,Incheon,인천광역시
4,"POLYGON ((126.76000 35.25900, 126.73600 35.251...",29,Gwangju,광주광역시


In [16]:
df_2.drop(17,inplace=True)

In [17]:
df_2['계'] = df_2['계'].astype(float)

In [18]:
df_2['비율'] = df_2['계']/df_2['계'].sum()

In [19]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 27 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   도시        17 non-null     object 
 1   지역        17 non-null     object 
 2   계         17 non-null     float64
 3   0 - 4세    17 non-null     float64
 4   5 - 9세    17 non-null     float64
 5   10 - 14세  17 non-null     float64
 6   15 - 19세  17 non-null     float64
 7   20 - 24세  17 non-null     float64
 8   25 - 29세  17 non-null     float64
 9   30 - 34세  17 non-null     float64
 10  35 - 39세  17 non-null     float64
 11  40 - 44세  17 non-null     float64
 12  45 - 49세  17 non-null     float64
 13  50 - 54세  17 non-null     float64
 14  55 - 59세  17 non-null     float64
 15  60 - 64세  17 non-null     float64
 16  65 - 69세  17 non-null     float64
 17  70 - 74세  17 non-null     float64
 18  75 - 79세  17 non-null     float64
 19  80세 이상    17 non-null     float64
 20  80 - 84세  17 non-null     float64


In [20]:
df_2

,도시,지역,계,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,...,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상,비율
0,서울특별시,수도권,9334933.5,220265.5,308678.0,357808.5,381243.0,564693.5,802843.5,764498.5,...,409781.0,311120.5,372713.5,222881.5,149832.0,105271.0,35320.5,7997.0,1243.5,0.182516
1,부산광역시,비수도권,3291326.0,78305.5,121997.0,133933.5,130393.0,177426.5,208199.5,200972.0,...,184261.0,129523.0,154773.0,94504.0,60269.0,43269.5,13677.5,2927.5,394.5,0.064352
2,대구광역시,비수도권,2372562.0,59547.0,94764.0,106649.0,109453.5,137540.5,150989.0,143435.5,...,113543.0,77986.0,106327.5,62908.5,43419.0,31388.5,9946.0,1837.5,247.0,0.046388
3,인천광역시,수도권,2966979.0,84134.0,121382.0,137201.0,134775.0,163436.0,205728.5,209312.0,...,110819.0,81399.0,106370.5,60135.5,46235.0,31607.0,11530.5,2665.5,432.0,0.058010
4,광주광역시,비수도권,1420562.0,40702.5,62653.5,73565.0,75537.0,93358.0,100970.0,88568.0,...,55494.0,41910.0,54074.0,31736.5,22337.5,15645.5,5395.5,1114.0,182.5,0.027775
5,대전광역시,비수도권,1438846.0,40665.0,58776.0,67868.5,71044.0,92828.5,108309.0,97767.5,...,56734.5,40211.0,54537.5,31267.0,23270.5,16261.5,5618.5,1198.5,192.0,0.028132
6,울산광역시,비수도권,1103825.0,31542.0,50497.0,56355.0,53417.5,57363.0,68305.5,68602.0,...,42883.0,26278.5,30852.5,18492.0,12360.5,8724.5,2944.0,624.0,68.0,0.021582
7,세종특별자치시,비수도권,384276.5,18091.0,26644.0,27710.0,22129.5,17677.5,20367.0,27577.0,...,9696.0,6348.0,9609.0,5109.5,4499.5,3023.5,1191.5,248.0,36.5,0.007513
8,경기도,수도권,13545542.5,414094.0,600617.5,671466.5,650517.0,772182.5,932864.0,943400.0,...,470060.5,354344.0,473734.5,269574.0,204160.5,142329.0,49266.5,10889.5,1675.5,0.264841
9,강원도,비수도권,1526763.0,39645.5,53774.5,63967.0,66508.0,81638.0,84848.0,80114.5,...,75982.0,63703.5,95323.5,54346.0,40977.5,28362.5,9848.5,2410.5,356.0,0.029851


In [21]:
# 시도별 인구 분포
fig = px.choropleth_mapbox(data_frame=df_2, geojson=geo_json,
                           featureidkey="properties.CTP_KOR_NM",
                           locations=df_2['도시'], color=df_2['비율']*5,
                           color_continuous_scale='Blues',
                           range_color=(0, 1),
                           opacity=0.5, zoom=5.5,
                           center={"lat": 36, "lon": 127.5},
                           mapbox_style="carto-positron"
                           )
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

In [22]:
df_2_group=df_2.groupby(['지역']).sum().reset_index()
df_2_group = df_2_group.reindex([1, 0])
df_2_group

<ipython-input-22-c0f5b3ccd782>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,지역,계,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,30 - 34세,35 - 39세,...,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상,비율
1,수도권,25847455.0,718493.5,1030677.5,1166476.0,1166535.0,1500312.0,1941436.0,1917210.5,1780791.0,...,990660.5,746863.5,952818.5,552591.0,400227.5,279207.0,96117.5,21552.0,3351.0,0.505367
0,비수도권,25298429.5,662311.0,1019557.5,1175043.0,1162747.0,1361339.5,1487487.5,1428073.5,1440575.0,...,1238376.0,906639.0,1344642.5,761377.5,583265.0,409359.0,140772.0,28741.0,4393.0,0.494633


In [23]:
df_2_group['10세 미만'] = df_2_group['0 - 4세'] + df_2_group['5 - 9세']
df_2_group['10대'] = df_2_group['10 - 14세'] + df_2_group['15 - 19세']
df_2_group['20대'] = df_2_group['20 - 24세'] + df_2_group['25 - 29세']
df_2_group['30대'] = df_2_group['30 - 34세'] + df_2_group['35 - 39세']
df_2_group['40대'] = df_2_group['40 - 44세'] + df_2_group['45 - 49세']
df_2_group['50대'] = df_2_group['50 - 54세'] + df_2_group['55 - 59세']
df_2_group['60대'] = df_2_group['60 - 64세'] + df_2_group['65 - 69세']
df_2_group['70대'] = df_2_group['70 - 74세'] + df_2_group['75 - 79세']
df_2_group['80대'] = df_2_group['80 - 84세'] + df_2_group['85 - 89세']
df_2_group['90대'] = df_2_group['90 - 94세'] + df_2_group['95 - 99세']

In [24]:
df_2_group = df_2_group[['지역','10세 미만','10대','20대','30대','40대','50대','60대','70대','80대','90대','100세 이상']]
df_2_group

,지역,10세 미만,10대,20대,30대,40대,50대,60대,70대,80대,90대,100세 이상
1,수도권,1749171.0,2333011.0,3441748.0,3698001.5,4155268.0,4280094.0,3499819.0,1737524.0,831798.0,117669.5,3351.0
0,비수도권,1681868.5,2337790.0,2848827.0,2868648.5,3790777.0,4308227.0,3972634.0,2145015.0,1170736.5,169513.0,4393.0


In [25]:
# 지역별/연령별 인구수
fig = go.Figure()

colors = {
    '10세 미만': 'skyblue',
    '10대': 'skyblue',
    '20대': 'rgb(100,100,300)',
    '30대': 'rgb(100,100,300)',
    '40대': 'rgb(100,100,300)',
    '50대': 'skyblue',
    '60대': 'rgb(150,150,450)',
    '70대': 'rgb(150,150,450)',
    '80대': 'rgb(150,150,450)',
    '90대': 'rgb(150,150,450)',
    '100세 이상': 'rgb(150,150,450)',
}

for column in df_2_group.columns[1:]:
    fig.add_trace(go.Bar(
        x=df_2_group['지역'],
        y=df_2_group[column],
        name=column,
        marker_color=colors[column],
        text=round(df_2_group[column]/1000000,1)
    ))

fig.update_layout(barmode='group')
fig.update_layout(
    autosize=False,
    width=1200,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

In [26]:
df_2_group['60세 미만'] = df_2_group[['10세 미만', '10대', '20대', '30대', '40대', '50대']].sum(axis=1)
df_2_group['60세 이상'] = df_2_group[['60대', '70대', '80대', '90대', '100세 이상']].sum(axis=1)
df_2_group

<ipython-input-26-f30ede39c531>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-26-f30ede39c531>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,지역,10세 미만,10대,20대,30대,40대,50대,60대,70대,80대,90대,100세 이상,60세 미만,60세 이상
1,수도권,1749171.0,2333011.0,3441748.0,3698001.5,4155268.0,4280094.0,3499819.0,1737524.0,831798.0,117669.5,3351.0,19657293.5,6190161.5
0,비수도권,1681868.5,2337790.0,2848827.0,2868648.5,3790777.0,4308227.0,3972634.0,2145015.0,1170736.5,169513.0,4393.0,17836138.0,7462291.5


In [27]:
df_2_group = df_2_group[['지역','60세 미만', '60세 이상']]

In [28]:
df_2_group_cap=df_2_group[df_2_group['지역']=='수도권']
df_2_group_oth=df_2_group[df_2_group['지역']=='비수도권']

In [29]:
df_2_group_cap

,지역,60세 미만,60세 이상
1,수도권,19657293.5,6190161.5


In [30]:
df_2_group_oth

,지역,60세 미만,60세 이상
0,비수도권,17836138.0,7462291.5


In [31]:
labels = ['60세 미만', '60세 이상']
values = [df_2_group_cap['60세 미만'].iloc[0], df_2_group_cap['60세 이상'].iloc[0]]
colors = ['rgb(150, 150, 450)', 'rgb(100, 100, 300)']

fig = go.Figure(data=[go.Pie(labels=labels, values=values,
                             marker=dict(colors=colors))])

fig.update_layout(barmode='group')
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()


In [32]:
labels = ['60세 미만', '60세 이상']
values = [df_2_group_oth['60세 미만'].iloc[0], df_2_group_oth['60세 이상'].iloc[0]]
colors = ['rgb(150, 150, 450)', 'rgb(100, 100, 300)']

fig = go.Figure(data=[go.Pie(labels=labels, values=values,
                             marker=dict(colors=colors))])

fig.update_layout(barmode='group')
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()


# 현황

## 경제

In [33]:
economy_path = 'https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/economy.xlsx'
economy_df = pd.read_excel(economy_path)
economy_df.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,시도별,2022 p),2022 p).1
0,시도별,1인당 지역총소득,1인당 개인소득
1,서울특별시,57236,26112
2,부산광역시,32293,22577
3,대구광역시,31056,22368
4,인천광역시,37442,22406


In [34]:
new_columns = economy_df.iloc[0]
economy_df.columns = new_columns
economy_df = economy_df.drop(0)

In [35]:
economy_df.reset_index(inplace=True)

In [36]:
economy_df.drop(columns='index', inplace=True)

In [37]:
economy_df.rename(columns={'시도별':'도시'}, inplace=True)

economy_df['지역']='비수도권'
economy_df.loc[economy_df['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'
economy_df.insert(0, '지역', economy_df.pop('지역'))

In [38]:
economy_df.head()

,지역,도시,1인당 지역총소득,1인당 개인소득
0,수도권,서울특별시,57236,26112
1,비수도권,부산광역시,32293,22577
2,비수도권,대구광역시,31056,22368
3,수도권,인천광역시,37442,22406
4,비수도권,광주광역시,34844,23669


In [39]:
economy_df_group = economy_df.groupby('지역').mean().reset_index()
economy_df_group = economy_df_group.reindex([1, 0]).reset_index()

<ipython-input-39-a4796531664e>:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [40]:
economy_df_group.drop(columns='index', inplace=True)

In [41]:
economy_df_group

,지역,1인당 지역총소득,1인당 개인소득
0,수도권,46104.333333,23884.666667
1,비수도권,38151.642857,22818.571429


In [42]:
# 지역별 1인당 평균 소득
fig = go.Figure()

fig.add_trace(go.Bar(x=economy_df_group['지역'],
                     y=economy_df_group['1인당 개인소득'],
                     text=round(economy_df_group['1인당 개인소득']),
                     marker_color='rgb(100,100,300)',
                     name='개인소득'))

fig.add_trace(go.Bar(x=economy_df_group['지역'],
                     y=(economy_df_group['1인당 지역총소득'] - economy_df_group['1인당 개인소득']),
                     text=round((economy_df_group['1인당 지역총소득'])),
                     marker_color='rgb(150,150,450)',
                     name='지역총소득'))

fig.update_layout(barmode='stack')

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

## 교육

In [43]:
edu_path='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/edu.csv'

edu_df=pd.read_csv(edu_path, encoding='cp949')
edu_df.head()

,학교명,학교 영문명,본분교구분명,대학구분명,학교구분명,설립형태구분명,시도코드,시도명,소재지도로명주소,소재지지번주소,도로명우편번호,소재지우편번호,홈페이지주소,대표전화번호,대표팩스번호,설립일자,기준연도,데이터기준일자,제공기관코드,제공기관명
0,한신대학교,Hanshin University,본교,대학,대학교,사립,41,경기도,"경기도 오산시 한신대길 137 (양산동, 한신대학교)",NaN,18101,NaN,www.hs.ac.kr,031-379-0114,031-379-0039,1940-04-19,2022,2023-12-27,B340014,한국대학교육협의회
1,한세대학교,Hansei University,본교,대학,대학교,사립,41,경기도,경기도 군포시 한세로 30 (당정동),NaN,15852,NaN,www.hansei.ac.kr,031-450-5114,031-450-5124,1953-05-10,2022,2023-12-27,B340014,한국대학교육협의회
2,성결대학교,Sungkyul University,본교,대학,대학교,사립,41,경기도,"경기도 안양시 만안구 성결대학로 53 (안양동, 성결대학교)",NaN,14097,NaN,www.sungkyul.ac.kr,031-467-8114,031-449-0529,1965-01-13,2022,2023-12-27,B340014,한국대학교육협의회
3,평택대학교,Pyeongtaek University,본교,대학,대학교,사립,41,경기도,"경기도 평택시 서동대로 3825 (용이동, 평택대학교)",NaN,17869,NaN,www.ptu.ac.kr,031-659-8114,031-659-8011,1990-11-28,2022,2023-12-27,B340014,한국대학교육협의회
4,성공회대학교,Sungkonghoe University,본교,대학,대학교,사립,11,서울특별시,"서울특별시 구로구 연동로 320 (항동, 성공회대학교)",NaN,8359,NaN,www.skhu.ac.kr,02-2610-4114,02-2610-4248,1992-12-23,2022,2023-12-27,B340014,한국대학교육협의회


In [44]:
edu_df_count = edu_df.groupby('시도명').agg(대학=pd.NamedAgg(column='대학구분명', aggfunc=lambda x: x.eq('대학').sum()),
                                     전문대학=pd.NamedAgg(column='대학구분명', aggfunc=lambda x: x.eq('전문대학').sum())).reset_index()

In [45]:
edu_df_count.rename(columns={'시도명':'도시'}, inplace=True)

In [46]:
edu_df_count['지역']='비수도권'
edu_df_count.loc[edu_df_count['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'
edu_df_count.insert(0, '지역', edu_df_count.pop('지역'))

In [47]:
edu_df_count['합계']=edu_df_count['대학']+edu_df_count['전문대학']

In [48]:
edu_df_count.loc[edu_df_count['도시'] == '강원특별자치도', '도시'] = '강원도'

In [49]:
edu_df_count

,지역,도시,대학,전문대학,합계
0,비수도권,강원도,14,13,27
1,수도권,경기도,40,37,77
2,비수도권,경상남도,11,14,25
3,비수도권,경상북도,22,20,42
4,비수도권,광주광역시,11,7,18
5,비수도권,대구광역시,4,10,14
6,비수도권,대전광역시,15,5,20
7,비수도권,부산광역시,17,12,29
8,수도권,서울특별시,57,12,69
9,비수도권,세종특별자치시,3,1,4


In [50]:
# 지역별 대학/전문대학 수
edu_df_count_cap=edu_df_count[edu_df_count['지역']=='수도권']
edu_df_count_oth=edu_df_count[edu_df_count['지역']=='비수도권']

fig = go.Figure()

fig.add_trace(go.Bar(x=edu_df_count_cap['도시'],
                     y=edu_df_count_cap['합계'],
                     name='수도권',
                     marker_color="rgb(100,100,300)",
                     text=edu_df_count_cap['합계']))

fig.add_trace(go.Bar(x=edu_df_count_oth['도시'],
                     y=edu_df_count_oth['합계'],
                     name='비수도권',
                     marker_color="rgb(150,150,450)",
                     text=edu_df_count_oth['합계']))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()


## 교통

In [51]:
trafic_path='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/trafic.xlsx'

trafic_df=pd.read_excel(trafic_path)
trafic_df

,시도코드,시도,년,교통수단,발생량,도착량
0,합계,NaN,NaN,NaN,7980291016,6967827119
1,11,서울특별시,2023.0,좌석,1917876,1850621
2,NaN,서울특별시,NaN,시내,1294222753,1265833029
3,NaN,서울특별시,NaN,마을,299405042,293266980
4,NaN,서울특별시,NaN,도시철도,1896284027,1899827451
...,...,...,...,...,...,...
74,51,강원특별자치도,2023.0,좌석,254956,59161
75,NaN,강원특별자치도,NaN,시내,22547474,9188182
76,NaN,강원특별자치도,NaN,마을,882634,381813
77,NaN,강원특별자치도,NaN,도시철도,1233575,1189884


In [52]:
trafic_df.drop(columns='시도코드' ,inplace=True)
trafic_df.drop(columns='도착량', inplace=True)
trafic_df.drop(columns='년', inplace=True)

In [53]:
trafic_df_group=trafic_df.groupby('시도')['발생량'].sum().reset_index()

In [54]:
trafic_df_group = trafic_df_group.append({'시도':'강원도',
                                          '발생량':trafic_df_group[trafic_df_group['시도']=='강원도']['발생량'].values[0]+trafic_df_group[trafic_df_group['시도']=='강원특별자치도']['발생량'].values[0]}, ignore_index=True)

<ipython-input-54-72bba936596a>:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [55]:
trafic_df_group.drop(0, inplace=True)
trafic_df_group.drop(1, inplace=True)

In [56]:
trafic_df_group.rename(columns={'시도':'도시'}, inplace=True)

In [57]:
trafic_df_group['지역']='비수도권'
trafic_df_group.loc[trafic_df_group['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'
trafic_df_group.insert(0, '지역', trafic_df_group.pop('지역'))

In [58]:
trafic_df_group=trafic_df_group.sort_values('도시').reset_index()

In [59]:
trafic_df_group.drop(columns='index', inplace=True)

In [60]:
trafic_df_group.rename(columns={'발생량':'대중교통이용량'}, inplace=True)

In [61]:
trafic_df_group

,지역,도시,대중교통이용량
0,비수도권,강원도,43955463
1,수도권,경기도,1833395164
2,비수도권,경상남도,165651483
3,비수도권,경상북도,91492652
4,비수도권,광주광역시,117909907
5,비수도권,대구광역시,301327052
6,비수도권,대전광역시,167168410
7,비수도권,부산광역시,765840871
8,수도권,서울특별시,3577353202
9,비수도권,세종특별자치시,19927506


In [62]:
trafic_df_group_cap=trafic_df_group[trafic_df_group['지역']=='수도권']
trafic_df_group_oth=trafic_df_group[trafic_df_group['지역']=='비수도권']

In [63]:
# 시도별 대중교통 이용 수
fig = go.Figure()

fig.add_trace(go.Bar(x=trafic_df_group_cap['도시'],
                     y=trafic_df_group_cap['대중교통이용량'],
                     name='수도권',
                     marker_color="rgb(100,100,300)",
                     text=round(trafic_df_group_cap['대중교통이용량']/1000000000,2)))

fig.add_trace(go.Bar(x=trafic_df_group_oth['도시'],
                     y=trafic_df_group_oth['대중교통이용량'],
                     name='비수도권',
                     marker_color="rgb(150,150,450)",
                     text=round(trafic_df_group_oth['대중교통이용량']/1000000000,2)))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()

## 인구

In [64]:
peo_path='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/people.xlsx'

peo_df = pd.read_excel(peo_path)
peo_df.head()

,시점,행정구역(시군구)별,성별,계,0 - 4세,5 - 9세,10 - 14세,15 - 19세,20 - 24세,25 - 29세,...,65 - 69세,70 - 74세,75 - 79세,80세 이상,80 - 84세,85세 이상,85 - 89세,90 - 94세,95 - 99세,100세 이상
0,2023.0,서울특별시,계,9334933.5,220265.5,308678.0,357808.5,381243.0,564693.5,802843.5,...,588956.0,409781.0,311120.5,372713.5,222881.5,149832.0,105271.0,35320.5,7997.0,1243.5
1,2023.0,부산광역시,계,3291326,78305.5,121997.0,133933.5,130393.0,177426.5,208199.5,...,258134.5,184261.0,129523.0,154773.0,94504.0,60269.0,43269.5,13677.5,2927.5,394.5
2,2023.0,대구광역시,계,2372562,59547.0,94764.0,106649.0,109453.5,137540.5,150989.0,...,157212.0,113543.0,77986.0,106327.5,62908.5,43419.0,31388.5,9946.0,1837.5,247.0
3,2023.0,인천광역시,계,2966979,84134.0,121382.0,137201.0,134775.0,163436.0,205728.5,...,179728.5,110819.0,81399.0,106370.5,60135.5,46235.0,31607.0,11530.5,2665.5,432.0
4,2023.0,광주광역시,계,1420562,40702.5,62653.5,73565.0,75537.0,93358.0,100970.0,...,76434.0,55494.0,41910.0,54074.0,31736.5,22337.5,15645.5,5395.5,1114.0,182.5


In [65]:
peo_df.rename(columns={'행정구역(시군구)별':'도시',
                       '계':'인구수'}, inplace=True)

In [66]:
peo_df=peo_df[['도시','인구수']]

In [67]:
peo_df['지역']='비수도권'
peo_df.loc[peo_df['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'
peo_df.insert(0, '지역', peo_df.pop('지역'))

<ipython-input-67-cc03e43e5f10>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [68]:
peo_df=peo_df.drop(17)

In [69]:
peo_df

,지역,도시,인구수
0,수도권,서울특별시,9334933.5
1,비수도권,부산광역시,3291326
2,비수도권,대구광역시,2372562
3,수도권,인천광역시,2966979
4,비수도권,광주광역시,1420562
5,비수도권,대전광역시,1438846
6,비수도권,울산광역시,1103825
7,비수도권,세종특별자치시,384276.5
8,수도권,경기도,13545542.5
9,비수도권,강원도,1526763


In [70]:
peo_df['인구수'] = peo_df['인구수'].astype(float)

In [71]:
peo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지역      17 non-null     object 
 1   도시      17 non-null     object 
 2   인구수     17 non-null     float64
dtypes: float64(1), object(2)
memory usage: 536.0+ bytes


In [72]:
peo_df['인구수'].sum()

51145884.5

In [73]:
# 지역별 인구수
peo_df_cap=peo_df[peo_df['지역']=='수도권']
peo_df_oth=peo_df[peo_df['지역']=='비수도권']

fig = go.Figure()

fig.add_trace(go.Bar(x=peo_df_cap['도시'],
                     y=peo_df_cap['인구수'],
                     name='수도권',
                     marker_color="rgb(100,100,300)",
                     text=round(peo_df_cap['인구수']/1000000,1)))

fig.add_trace(go.Bar(x=peo_df_oth['도시'],
                     y=peo_df_oth['인구수'],
                     name='비수도권',
                     marker_color="rgb(150,150,450)",
                     text=round(peo_df_oth['인구수']/1000000,1)))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()


## 의료

In [74]:
med_path='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/med.xlsx'

med_df=pd.read_excel(med_path)
med_df

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,시도별(1),2022
0,시도별(1),총계
1,시도별(1),기관수 (개소)
2,서울특별시,26
3,부산광역시,10
4,대구광역시,9
5,인천광역시,8
6,광주광역시,10
7,대전광역시,8
8,울산광역시,1
9,세종특별자치시,1


In [75]:
med_df.rename(columns={'시도별(1)':'도시',
                         '2022':'의료기관수'}, inplace=True)

In [76]:
med_df.drop(0, inplace=True)

In [77]:
med_df.drop(1, inplace=True)

In [78]:
med_df=med_df.reset_index()

In [79]:
med_df=med_df.drop(columns='index')

In [80]:
med_df['지역']='비수도권'
med_df.loc[med_df['도시'].isin(['서울특별시', '인천광역시', '경기도']), '지역'] = '수도권'
med_df.insert(0, '지역', med_df.pop('지역'))

In [81]:
med_df.loc[med_df['도시'] == '강원특별자치도', '도시'] = '강원도'

In [82]:
med_df

,지역,도시,의료기관수
0,수도권,서울특별시,26
1,비수도권,부산광역시,10
2,비수도권,대구광역시,9
3,수도권,인천광역시,8
4,비수도권,광주광역시,10
5,비수도권,대전광역시,8
6,비수도권,울산광역시,1
7,비수도권,세종특별자치시,1
8,수도권,경기도,32
9,비수도권,강원도,18


In [83]:
med_df['의료기관수'].sum()

231

In [84]:
# 지역별 공공의료기관 수
med_df_cap=med_df[med_df['지역']=='수도권']
med_df_oth=med_df[med_df['지역']=='비수도권']

fig = go.Figure()

fig.add_trace(go.Bar(x=med_df_cap['도시'],
                     y=med_df_cap['의료기관수'],
                     name='수도권',
                     marker_color="rgb(100,100,300)",
                     text=med_df_cap['의료기관수']))

fig.add_trace(go.Bar(x=med_df_oth['도시'],
                     y=med_df_oth['의료기관수'],
                     name='비수도권',
                     marker_color="rgb(150,150,450)",
                     text=med_df_oth['의료기관수']))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()


## 주거

In [85]:
house_path='https://raw.githubusercontent.com/UBDBD/Regional-deviation-analysis/main/Data/house.xlsx'

house_df = pd.read_excel(house_path)
house_df.head()

/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



,시도별(1),시도별(2),2023.01,2023.02,2023.03,2023.04,2023.05,2023.06,2023.07,2023.08,2023.09,2023.10,2023.11,2023.12
0,수도권,소계,83.2,94.1,97.0,99.0,103.3,104.3,108.7,111.8,113.4,106.2,99.2,96.8
1,수도권,서울특별시,82.1,93.3,97.7,100.0,104.6,107.2,111.1,114.8,116.9,110.3,101.6,98.4
2,수도권,인천광역시,84.7,95.6,94.4,95.0,97.1,100.1,102.9,104.4,107.7,101.8,96.1,96.2
3,수도권,경기도,83.4,94.2,97.1,99.3,104.0,103.5,108.7,111.7,112.6,104.7,98.4,96.0
4,비수도권,소계,84.7,93.5,94.0,97.7,99.4,101.8,103.3,106.6,107.7,103.2,97.5,96.1


In [86]:
house_df=house_df.rename(columns={'시도별(1)':'지역',
                        '시도별(2)':'도시'})

In [87]:
house_df['평균']=house_df.mean(axis='columns')
house_df.insert(2, '평균', house_df.pop('평균'))

<ipython-input-87-cd4743f67bab>:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [88]:
house_df.drop([0,4], axis=0, inplace=True)

In [89]:
house_df.loc[house_df['도시'] == '강원특별자치도', '도시'] = '강원도'

In [90]:
house_df

,지역,도시,평균,2023.01,2023.02,2023.03,2023.04,2023.05,2023.06,2023.07,2023.08,2023.09,2023.10,2023.11,2023.12
1,수도권,서울특별시,103.166667,82.1,93.3,97.7,100.0,104.6,107.2,111.1,114.8,116.9,110.3,101.6,98.4
2,수도권,인천광역시,98.000000,84.7,95.6,94.4,95.0,97.1,100.1,102.9,104.4,107.7,101.8,96.1,96.2
3,수도권,경기도,101.133333,83.4,94.2,97.1,99.3,104.0,103.5,108.7,111.7,112.6,104.7,98.4,96.0
5,비수도권,부산광역시,94.300000,80.0,88.6,89.0,93.7,95.3,95.1,101.0,101.2,101.7,98.5,94.6,92.9
6,비수도권,대구광역시,92.316667,77.7,85.7,87.5,91.7,93.3,95.9,96.7,104.1,103.0,95.6,88.4,88.2
7,비수도권,광주광역시,97.416667,83.4,91.9,92.4,97.9,98.7,102.9,100.4,105.2,106.5,100.0,96.0,93.7
8,비수도권,대전광역시,101.041667,87.1,94.6,96.3,96.8,97.9,105.9,108.0,111.7,110.7,106.2,98.4,98.9
9,비수도권,울산광역시,100.483333,83.8,95.8,95.4,99.9,100.9,105.9,106.1,107.7,108.4,106.9,97.7,97.3
10,비수도권,세종특별자치시,105.691667,83.4,100.5,96.5,107.1,110.8,115.6,112.9,119.6,122.7,107.1,99.0,93.1
11,비수도권,강원도,106.066667,90.8,95.4,102.3,102.1,105.3,107.7,112.7,117.3,118.5,109.1,105.1,106.5


In [91]:
house_df_cap=house_df[house_df['지역']=='수도권']

In [92]:
house_df_oth=house_df[house_df['지역']=='비수도권']

In [93]:
# 시도별 주택소비심리지수
fig=go.Figure()

fig.add_trace(go.Bar(x=house_df_cap['도시'],
                     y=house_df_cap['평균'],
                     name='수도권',
                     marker_color='rgb(100, 100, 300)',
                     text=round(house_df_cap['평균'])))

fig.add_trace(go.Bar(x=house_df_oth['도시'],
                     y=house_df_oth['평균'],
                     name='비수도권',
                     marker_color='rgb(150,150,450)',
                     text=round(house_df_oth['평균'])))

fig.update_yaxes(range=[50, 150])
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)


fig.show()

# 관계

In [94]:
house_df=house_df.rename(columns={'평균':'주택소비심리지수'})

In [95]:
house_df=house_df[['지역','도시','주택소비심리지수']]

In [96]:
house_df.reset_index(inplace=True)
house_df.drop(columns='index', inplace=True)

In [97]:
house_df

,지역,도시,주택소비심리지수
0,수도권,서울특별시,103.166667
1,수도권,인천광역시,98.000000
2,수도권,경기도,101.133333
3,비수도권,부산광역시,94.300000
4,비수도권,대구광역시,92.316667
5,비수도권,광주광역시,97.416667
6,비수도권,대전광역시,101.041667
7,비수도권,울산광역시,100.483333
8,비수도권,세종특별자치시,105.691667
9,비수도권,강원도,106.066667


In [98]:
economy_df

,지역,도시,1인당 지역총소득,1인당 개인소득
0,수도권,서울특별시,57236,26112
1,비수도권,부산광역시,32293,22577
2,비수도권,대구광역시,31056,22368
3,수도권,인천광역시,37442,22406
4,비수도권,광주광역시,34844,23669
5,비수도권,대전광역시,35848,24220
6,비수도권,울산광역시,60971,26066
7,비수도권,세종특별자치시,41372,23215
8,수도권,경기도,43635,23136
9,비수도권,강원도,33198,22395


In [99]:
trafic_df_group.rename(columns={'발생량':'대중교통이용량'}, inplace=True)

In [100]:
trafic_df_group

,지역,도시,대중교통이용량
0,비수도권,강원도,43955463
1,수도권,경기도,1833395164
2,비수도권,경상남도,165651483
3,비수도권,경상북도,91492652
4,비수도권,광주광역시,117909907
5,비수도권,대구광역시,301327052
6,비수도권,대전광역시,167168410
7,비수도권,부산광역시,765840871
8,수도권,서울특별시,3577353202
9,비수도권,세종특별자치시,19927506


In [101]:
edu_df_count.drop(columns='합계', inplace=True)

In [102]:
edu_df_count

,지역,도시,대학,전문대학
0,비수도권,강원도,14,13
1,수도권,경기도,40,37
2,비수도권,경상남도,11,14
3,비수도권,경상북도,22,20
4,비수도권,광주광역시,11,7
5,비수도권,대구광역시,4,10
6,비수도권,대전광역시,15,5
7,비수도권,부산광역시,17,12
8,수도권,서울특별시,57,12
9,비수도권,세종특별자치시,3,1


In [103]:
geo_df

,geometry,CTPRVN_CD,CTP_ENG_NM,CTP_KOR_NM
0,"POLYGON ((126.98400 37.63600, 126.94800 37.657...",11,Seoul,서울특별시
1,"POLYGON ((129.28800 35.32100, 129.26300 35.386...",26,Busan,부산광역시
2,"POLYGON ((128.47300 35.83300, 128.47000 35.806...",27,Daegu,대구광역시
3,"MULTIPOLYGON (((126.34300 37.64400, 126.37500 ...",28,Incheon,인천광역시
4,"POLYGON ((126.76000 35.25900, 126.73600 35.251...",29,Gwangju,광주광역시
5,"POLYGON ((127.40200 36.48600, 127.39800 36.490...",30,Daejeon,대전광역시
6,"POLYGON ((129.34600 35.46500, 129.40800 35.493...",31,Ulsan,울산광역시
7,"POLYGON ((127.17800 36.59700, 127.19400 36.565...",36,Sejong-si,세종특별자치시
8,"POLYGON ((127.12700 37.46900, 127.07100 37.432...",41,Gyeonggi-do,경기도
9,"POLYGON ((128.54900 38.30200, 128.51300 38.346...",42,Gangwon-do,강원도


In [104]:
peo_df=peo_df[['지역','도시','인구수']]

In [105]:
#peo_df.rename(columns={'계':'인구수'}, inplace=True)

In [106]:
peo_df

,지역,도시,인구수
0,수도권,서울특별시,9334933.5
1,비수도권,부산광역시,3291326.0
2,비수도권,대구광역시,2372562.0
3,수도권,인천광역시,2966979.0
4,비수도권,광주광역시,1420562.0
5,비수도권,대전광역시,1438846.0
6,비수도권,울산광역시,1103825.0
7,비수도권,세종특별자치시,384276.5
8,수도권,경기도,13545542.5
9,비수도권,강원도,1526763.0


In [107]:
df_1=df_1[df_1['시점']==2023]

In [108]:
df_1.rename(columns={'순이동':'인구순이동'}, inplace=True)

<ipython-input-108-74c5afb17cfe>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [109]:
df_1.drop(columns='시점', inplace=True)

<ipython-input-109-771fc3b8acc2>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [110]:
df_1

,지역,도시,인구순이동
153,수도권,서울특별시,-31250
154,비수도권,부산광역시,-11432
155,비수도권,대구광역시,-5288
156,수도권,인천광역시,33507
157,비수도권,광주광역시,-9017
158,비수도권,대전광역시,-2474
159,비수도권,울산광역시,-6215
160,비수도권,세종특별자치시,1690
161,수도권,경기도,44612
162,비수도권,강원특별자치도,-1397


In [111]:
merge_df = pd.merge(peo_df, trafic_df_group, on=['지역','도시'])

In [112]:
merge_df = pd.merge(merge_df, edu_df_count, on=['지역','도시'])

In [113]:
merge_df = pd.merge(merge_df, med_df, on=['지역','도시'])

In [114]:
merge_df = pd.merge(merge_df, house_df, on=['지역','도시'])

In [115]:
merge_df = pd.merge(merge_df, economy_df, on=['지역','도시'])

In [116]:
merge_df

,지역,도시,인구수,대중교통이용량,대학,전문대학,의료기관수,주택소비심리지수,1인당 지역총소득,1인당 개인소득
0,수도권,서울특별시,9334933.5,3577353202,57,12,26,103.166667,57236,26112
1,비수도권,부산광역시,3291326.0,765840871,17,12,10,94.300000,32293,22577
2,비수도권,대구광역시,2372562.0,301327052,4,10,9,92.316667,31056,22368
3,수도권,인천광역시,2966979.0,479459277,6,6,8,98.000000,37442,22406
4,비수도권,광주광역시,1420562.0,117909907,11,7,10,97.416667,34844,23669
5,비수도권,대전광역시,1438846.0,167168410,15,5,8,101.041667,35848,24220
6,비수도권,울산광역시,1103825.0,80092990,2,3,1,100.483333,60971,26066
7,비수도권,세종특별자치시,384276.5,19927506,3,1,1,105.691667,41372,23215
8,수도권,경기도,13545542.5,1833395164,40,37,32,101.133333,43635,23136
9,비수도권,강원도,1526763.0,43955463,14,13,18,106.066667,33198,22395


In [117]:
def convert_to_int(value):
    try:
        return int(value)
    except ValueError:
        return None

convertible_columns = ['1인당 지역총소득', '1인당 개인소득','의료기관수','인구수']
for column in convertible_columns:
    merge_df[column] = merge_df[column].apply(convert_to_int)

In [118]:
merge_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 0 to 16
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   지역         17 non-null     object 
 1   도시         17 non-null     object 
 2   인구수        17 non-null     int64  
 3   대중교통이용량    17 non-null     int64  
 4   대학         17 non-null     int64  
 5   전문대학       17 non-null     int64  
 6   의료기관수      17 non-null     int64  
 7   주택소비심리지수   17 non-null     float64
 8   1인당 지역총소득  17 non-null     int64  
 9   1인당 개인소득   17 non-null     int64  
dtypes: float64(1), int64(7), object(2)
memory usage: 2.0+ KB


## 상관분석

In [119]:
corr_matrix = merge_df.corr()

<ipython-input-119-7d3d8a8e49e8>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [120]:
corr_matrix

,인구수,대중교통이용량,대학,전문대학,의료기관수,주택소비심리지수,1인당 지역총소득,1인당 개인소득
인구수,1.000000,0.827751,0.832235,0.805679,0.725838,0.079880,0.331336,0.272140
대중교통이용량,0.827751,1.000000,0.891809,0.416184,0.533731,0.099552,0.484686,0.538001
대학,0.832235,0.891809,1.000000,0.604727,0.747302,0.264346,0.423272,0.395635
전문대학,0.805679,0.416184,0.604727,1.000000,0.847350,0.021264,-0.025017,-0.151790
의료기관수,0.725838,0.533731,0.747302,0.847350,1.000000,0.212652,0.021255,-0.075340
주택소비심리지수,0.079880,0.099552,0.264346,0.021264,0.212652,1.000000,0.390483,0.230152
1인당 지역총소득,0.331336,0.484686,0.423272,-0.025017,0.021255,0.390483,1.000000,0.791304
1인당 개인소득,0.272140,0.538001,0.395635,-0.151790,-0.075340,0.230152,0.791304,1.000000


In [121]:
trace = go.Heatmap(z=corr_matrix.values,
                   x=corr_matrix.columns,
                   y=corr_matrix.index,
                   colorscale='blues')

fig = go.Figure(data=[trace])

annotations = []
for i, row in enumerate(corr_matrix.values):
    for j, value in enumerate(row):
        annotations.append(
            dict(
                x=corr_matrix.columns[j],
                y=corr_matrix.index[i],
                text=str(round(value, 2)),
                showarrow=False
            )
        )


fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
    annotations=annotations
)

fig.show()

In [122]:
corr_matrix['인구수']

인구수          1.000000
대중교통이용량      0.827751
대학           0.832235
전문대학         0.805679
의료기관수        0.725838
주택소비심리지수     0.079880
1인당 지역총소득    0.331336
1인당 개인소득     0.272140
Name: 인구수, dtype: float64

## 회귀분석

In [123]:
X = merge_df[['주택소비심리지수',	'1인당 지역총소득',	'1인당 개인소득',	'대중교통이용량',	'대학',	'전문대학','의료기관수']]
y = merge_df['인구수']

X = sm.add_constant(X)

StatsModels_model = sm.OLS(y, X).fit()

print(StatsModels_model.summary())

                            OLS Regression Results                            
Dep. Variable:                    인구수   R-squared:                       0.968
Model:                            OLS   Adj. R-squared:                  0.943
Method:                 Least Squares   F-statistic:                     38.97
Date:                Thu, 14 Mar 2024   Prob (F-statistic):           5.04e-06
Time:                        08:01:32   Log-Likelihood:                -249.81
No. Observations:                  17   AIC:                             515.6
Df Residuals:                       9   BIC:                             522.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -6.134e+06   8.62e+06     -0.712      0.4

/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:1806: UserWarning:

kurtosistest only valid for n>=20 ... continuing anyway, n=17



In [124]:
predicted_values = StatsModels_model.predict(X)

results_df = pd.DataFrame({'Actual': y, 'Predict': predicted_values})

fig = px.scatter(results_df, x='Actual', y='Predict')

fig.add_scatter(x=results_df['Actual'], y=results_df['Actual'], mode='lines', name='회귀선')

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()


# 지역편차

In [125]:
rat_df = merge_df.iloc[:, 2:].apply(lambda x: x / x.sum() * 100)

In [126]:
rat_df = round(rat_df * (corr_matrix['인구수']),2)

In [127]:
rat_df = pd.concat([merge_df.iloc[:, :2], rat_df], axis=1)

In [128]:
rat_df.sum(axis=1)

<ipython-input-128-460515e10f31>:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



0     92.10
1     31.86
2     19.84
3     21.72
4     17.58
5     17.97
6     10.56
7      6.83
8     88.96
9     23.05
10    18.35
11    23.08
12    20.26
13    23.49
14    33.90
15    28.16
16     9.78
dtype: float64

In [129]:
rat_df['지역편차'] = round(rat_df.sum(axis=1), 2)

<ipython-input-129-20699196306e>:1: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



In [130]:
rat_df

,지역,도시,인구수,대중교통이용량,대학,전문대학,의료기관수,주택소비심리지수,1인당 지역총소득,1인당 개인소득,지역편차
0,수도권,서울특별시,18.25,37.11,18.11,5.34,8.17,0.48,2.82,1.82,92.10
1,비수도권,부산광역시,6.44,7.94,5.40,5.34,3.14,0.44,1.59,1.57,31.86
2,비수도권,대구광역시,4.64,3.13,1.27,4.45,2.83,0.43,1.53,1.56,19.84
3,수도권,인천광역시,5.80,4.97,1.91,2.67,2.51,0.46,1.84,1.56,21.72
4,비수도권,광주광역시,2.78,1.22,3.49,3.12,3.14,0.46,1.72,1.65,17.58
5,비수도권,대전광역시,2.81,1.73,4.76,2.23,2.51,0.47,1.77,1.69,17.97
6,비수도권,울산광역시,2.16,0.83,0.64,1.34,0.31,0.47,3.00,1.81,10.56
7,비수도권,세종특별자치시,0.75,0.21,0.95,0.45,0.31,0.50,2.04,1.62,6.83
8,수도권,경기도,26.48,19.02,12.71,16.47,10.05,0.47,2.15,1.61,88.96
9,비수도권,강원도,2.99,0.46,4.45,5.79,5.66,0.50,1.64,1.56,23.05


In [131]:
rat_df=rat_df.sort_values('지역편차', ascending=False)

In [132]:
fig = go.Figure()

fig.add_trace(go.Bar(x=rat_df['도시'],
                     y=rat_df['지역편차'],
                     marker_color='rgb(100,100,300)',
                     name='지역편차',
                     text=round(rat_df['지역편차'])))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)
fig.update_layout(showlegend=True)

fig.show()


# 지역선정

In [133]:
select_df = merge_df[(merge_df['지역'] == '비수도권')]

In [134]:
selec_df=select_df[['도시','대중교통이용량','전문대학','인구수']]

In [135]:
selec_df_rat = selec_df.iloc[:, 1:].apply(lambda x: x / x.sum() * 100)

In [136]:
selec_df_rat.iloc[:, 0] = selec_df_rat.iloc[:, 0] * 0.83
selec_df_rat.iloc[:, 2] = selec_df_rat.iloc[:, 2] * 0.81

In [137]:
selec_df_rat['점수'] = selec_df_rat.sum(axis=1)

In [138]:
selec_df_rat = pd.concat([merge_df.iloc[:, :2], selec_df_rat], axis=1)

In [139]:
selec_df_rat.dropna(inplace=True)

In [140]:
selec_df_rat.sort_values('점수', ascending=False, inplace=True)

In [141]:
selec_df_rat

,지역,도시,대중교통이용량,전문대학,인구수,점수
1,비수도권,부산광역시,30.412563,9.523810,10.538102,50.474474
15,비수도권,경상남도,6.578241,11.111111,10.423270,28.112622
14,비수도권,경상북도,3.633295,15.873016,8.190835,27.697146
2,비수도권,대구광역시,11.966099,7.936508,7.596421,27.499029
11,비수도권,충청남도,3.964315,7.142857,6.787613,17.894786
13,비수도권,전라남도,2.459754,8.730159,5.784982,16.974895
9,비수도권,강원도,1.745530,10.317460,4.888359,16.951350
12,비수도권,전라북도,2.467988,8.730159,5.625659,16.823805
5,비수도권,대전광역시,6.638481,3.968254,4.606868,15.213603
4,비수도권,광주광역시,4.682360,5.555556,4.548327,14.786242


In [142]:
fig = go.Figure()

fig.add_trace(go.Bar(x=selec_df_rat['도시'],
                     y=selec_df_rat['점수'],
                     marker_color='rgb(100,100,300)',
                     name='점수',
                     text=round(selec_df_rat['점수'])))

fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)
fig.update_layout(showlegend=True)

fig.show()
